In [ ]:
import pandas as pd

df=pd.read_csv('data/listings_inside_airbnb.csv')
print(df.columns)
print(df.shape)
df.head(50)

In [ ]:
airbnb = df.drop(columns=['id', 'name', 'listing_url','scrape_id','last_scraped', 'source', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_location', 'host_response_time', 'host_acceptance_rate', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_total_listings_count', 'host_verifications', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'minimum_nights', 'bathrooms_text', 'maximum_nights', 'maximum_minimum_nights', 'minimum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_accuracy', 'review_scores_value', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month', 'license'])
print(airbnb.shape)
airbnb

Data cleaning

In [ ]:
airbnb.isnull().sum()/len(airbnb)*100

In [ ]:
import matplotlib.pyplot as plt

missing_values = airbnb.isnull().sum()

plt.figure(figsize=(10, 6))
missing_values.plot(kind='bar', color='skyblue')
plt.title('Number of Missing Values per Column', fontsize=16)
plt.xlabel('Columns', fontsize=14)
plt.ylabel('Number of Missing Values', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
airbnb = airbnb.dropna(subset=['price'])
print(airbnb.isnull().sum()/len(airbnb)*100)
airbnb.shape

In [ ]:
airbnb['host_identity_verified'].value_counts()

In [ ]:
# Drop columns where's not much data or data is not valuable
airbnb = airbnb.drop(columns=['neighborhood_overview', 'host_about', 'host_identity_verified'])

In [ ]:
print(f"Number of duplicates before removal operation: {airbnb.duplicated().sum()}")
airbnb = airbnb.drop_duplicates()
print(f"Number of duplicates after removal operation: {airbnb.duplicated().sum()}")

In [ ]:
import ast

# Convert price from string to number
airbnb['price'] = airbnb['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().astype(float)

# Convert date to number of dates
airbnb['host_since'] = pd.to_datetime(airbnb['host_since'])
today = pd.to_datetime('today')
airbnb['host_since'] = (today - airbnb['host_since']).dt.days

# Create a new column 'amenities_count' with the count of items in the list
airbnb['amenities_count'] = airbnb['amenities'].apply(lambda x: len(ast.literal_eval(x)))
airbnb = airbnb.drop(columns=['amenities'])

# Convert string to number and fill empty values with mean
airbnb['host_response_rate'] = airbnb['host_response_rate'].str.rstrip('%').astype('float') / 100
mean = airbnb['host_response_rate'].mean()
print(f"Filling {airbnb['host_response_rate'].isna().sum()} values with mean {mean}")
airbnb['host_response_rate'] = airbnb['host_response_rate'].fillna(mean)

In [ ]:
airbnb['price'].describe()

In [ ]:
# Fill ratings with mean values
mean = airbnb['review_scores_rating'].mean()
print(f"Filling {airbnb['review_scores_rating'].isna().sum()} values with mean {mean}")
airbnb['review_scores_rating'] = airbnb['review_scores_rating'].fillna(mean)

mean = airbnb['review_scores_cleanliness'].mean()
print(f"Filling {airbnb['review_scores_cleanliness'].isna().sum()} values with mean {mean}")
airbnb['review_scores_cleanliness'] = airbnb['review_scores_cleanliness'].fillna(mean)

mean = airbnb['review_scores_checkin'].mean()
print(f"Filling {airbnb['review_scores_checkin'].isna().sum()} values with mean {mean}")
airbnb['review_scores_checkin'] = airbnb['review_scores_checkin'].fillna(mean)

mean = airbnb['review_scores_communication'].mean()
print(f"Filling {airbnb['review_scores_communication'].isna().sum()} values with mean {mean}")
airbnb['review_scores_communication'] = airbnb['review_scores_communication'].fillna(mean)

mean = airbnb['review_scores_location'].mean()
print(f"Filling {airbnb['review_scores_location'].isna().sum()} values with mean {mean}")
airbnb['review_scores_location'] = airbnb['review_scores_location'].fillna(mean)

In [ ]:
print(f"Null elements count: {airbnb.isna().sum().sum()}")
airbnb.shape

In [ ]:
airbnb = airbnb.dropna()
print(f"Null elements count: {airbnb.isna().sum().sum()}")
print(f"Final shape {airbnb.shape}")

Feature engineering

In [ ]:
# Distance to attractions
nyc_attractions = [
    {"name": "Statue of Liberty", "latitude": 40.6892, "longitude": -74.0445},
    {"name": "Central Park", "latitude": 40.7851, "longitude": -73.9683},
    {"name": "Times Square", "latitude": 40.7580, "longitude": -73.9855},
    {"name": "Empire State Building", "latitude": 40.7484, "longitude": -73.9857},
    {"name": "Brooklyn Bridge", "latitude": 40.7061, "longitude": -73.9969},
    {"name": "Metropolitan Museum of Art", "latitude": 40.7794, "longitude": -73.9632},
    {"name": "One World Trade Center", "latitude": 40.7127, "longitude": -74.0134},
    {"name": "Rockefeller Center", "latitude": 40.7587, "longitude": -73.9787},
    {"name": "Broadway", "latitude": 40.7590, "longitude": -73.9845},
    {"name": "Fifth Avenue", "latitude": 40.7750, "longitude": -73.9650}
]

import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def calculate_avg_distance(row, attractions):
    distances = []
    for attraction in attractions:
        dist = haversine(row['latitude'], row['longitude'], attraction['latitude'], attraction['longitude'])
        distances.append(dist)
    return np.mean(distances)

airbnb['avg_distance_to_attractions'] = airbnb.apply(lambda row: calculate_avg_distance(row, nyc_attractions), axis=1)

In [ ]:
# Sentiment anlaysis
from textblob import TextBlob

def calculate_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

airbnb['description_sentiment'] = airbnb['description'].apply(calculate_sentiment)
airbnb['description_length'] = airbnb['description'].apply(len)
descriptions_merged = " ".join(airbnb['description'])
airbnb = airbnb.drop(columns=['description'])

In [ ]:
# Dataframe after cleaning and features engineering
airbnb.head()

In [ ]:
# Columns available after features engineering
airbnb.columns

Exploratory data analysis

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

custom_stopwords = set(STOPWORDS)
custom_stopwords.update(['br', 'New York', 'New', 'York', 'Manhattan'])  # Add more if needed

wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    stopwords=custom_stopwords,
    colormap='viridis',
    max_words=200
).generate(descriptions_merged)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Hide axes
plt.title("Word Cloud for Descriptions", fontsize=16)
plt.show()

In [ ]:
import folium
from folium.plugins import MarkerCluster

nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

marker_cluster = MarkerCluster().add_to(nyc_map)

for idx, row in airbnb.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], 
                  popup=f"Price: ${row['price']}\n Avg. dist. to attractions: {row['avg_distance_to_attractions']:.2f}km\n Accommodates: {row['accommodates']}",
                  icon=folium.Icon(color='blue', icon='home')).add_to(marker_cluster)

nyc_map.save('nyc_airbnb_map.html')
nyc_map

In [ ]:
import geopandas as gpd

nyc_shapefile = gpd.read_file('data/geo_export_4bd92b16-3dab-4b86-80ae-a5283a9caffa.shp')

neighbourhood_avg_price = airbnb.groupby('neighbourhood_group_cleansed')['price'].mean().reset_index()

nyc_geo = nyc_shapefile.merge(neighbourhood_avg_price, left_on='boro_name', right_on='neighbourhood_group_cleansed')
nyc_geo.plot(column='price', cmap='Wistia', legend=True, figsize=(10, 8))
plt.title('Average Price by Neighborhood Group')
plt.show()

In [ ]:
airbnb.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(airbnb['price'], bins=50, kde=True, color='blue')
plt.title('Distribution of Listing Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
lower_percentile = airbnb['price'].quantile(0.01)  # 1st percentile
upper_percentile = airbnb['price'].quantile(0.99)  # 99th percentile

airbnb = airbnb[(airbnb['price'] >= lower_percentile) & (airbnb['price'] <= upper_percentile)]
plt.figure(figsize=(10, 6))
sns.histplot(airbnb['price'], bins=50, kde=True, color='blue')
plt.title('Distribution of Listing Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='room_type', y='price', data=airbnb, hue='room_type', showfliers=False)
plt.title('Price Distribution by Room Type')
plt.xlabel('Room Type')
plt.ylabel('Price')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_histograms(dataframe, columns, bins=50, figsize=(12, 32)):
    n_cols = 2
    n_rows = -(-len(columns) // n_cols)

    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    for i, column in enumerate(columns):
        if column in dataframe.columns:
            axes[i].hist(dataframe[column].dropna(), bins=bins, color='blue', alpha=0.7, edgecolor='black')
            axes[i].set_title(column)
            axes[i].set_xlabel('Value')
            axes[i].set_ylabel('Frequency')
        else:
            axes[i].text(0.5, 0.5, f"Column '{column}' not found",
                         ha='center', va='center', fontsize=10, color='red')
            axes[i].set_axis_off()

    
    for j in range(len(columns), len(axes)):
        axes[j].set_axis_off()

    plt.tight_layout()
    plt.show()
    
def plot_violin_plots(dataframe, columns, figsize=(12, 32)):
    n_cols = 2
    n_rows = -(-len(columns) // n_cols)

    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    for i, column in enumerate(columns):
        sns.violinplot(x=dataframe[column], ax=axes[i])
        axes[i].set_title(column)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency')

    
    for j in range(len(columns), len(axes)):
        axes[j].set_axis_off()

    plt.tight_layout()
    plt.show()
    
def plot_box_plots(dataframe, columns, figsize=(12, 32)):
    n_cols = 2
    n_rows = -(-len(columns) // n_cols)

    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    for i, column in enumerate(columns):
        sns.boxplot(x=dataframe[column], ax=axes[i])
        axes[i].set_title(column)
        axes[i].set_xlabel('Value')

    
    for j in range(len(columns), len(axes)):
        axes[j].set_axis_off()

    plt.tight_layout()
    plt.show()

def plot_pie_charts_grid(dataframe, columns, figsize=(12, 8)):
    n_cols = 2
    n_rows = -(-len(columns) // n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    for i, column in enumerate(columns):
        if column in dataframe.columns:
            if column == 'bedrooms':
                data = dataframe[column].apply(lambda x: x if x <= 3 else '4+').value_counts()
            elif column == 'bathrooms':
                data = dataframe[column].apply(lambda x: x if x <= 2 else '3+').value_counts()
            else:
                data = dataframe[column].value_counts()

            labels = data.index
            sizes = data.values

            wedges, texts, autotexts = axes[i].pie(
                sizes, autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors, labels=None
            )

            axes[i].legend(wedges, labels, title=column, loc="best", fontsize='small')
            axes[i].set_title(f"Pie Chart of {column}")
        else:
            axes[i].text(0.5, 0.5, f"Column '{column}' not found",
                         ha='center', va='center', fontsize=10, color='red')
            axes[i].set_axis_off()

    for j in range(len(columns), len(axes)):
        axes[j].set_axis_off()

    plt.tight_layout()
    plt.show()


columns_to_plot_histogram = ['host_listings_count', 'avg_distance_to_attractions', 'number_of_reviews', 'bathrooms', 'bedrooms', 'beds']

columns_to_plot_violin_plots = ['host_since', 'accommodates', 'amenities_count', 'avg_distance_to_attractions']

columns_to_plot_box_plots = ['host_response_rate', 'review_scores_rating', 'description_sentiment', 'description_length']

plot_histograms(airbnb, columns_to_plot_histogram, figsize=(12, 12))
plot_violin_plots(airbnb, columns_to_plot_violin_plots, figsize=(12, 12))
plot_box_plots(airbnb, columns_to_plot_box_plots, figsize=(12, 12))

columns_to_plot_pie = ['room_type', 'neighbourhood_group_cleansed', 'host_is_superhost', 'host_has_profile_pic']

plot_pie_charts_grid(airbnb, columns_to_plot_pie)

In [ ]:
airbnb = airbnb.drop(columns = ['host_has_profile_pic', 'host_response_rate', 'host_listings_count'])

In [ ]:
upper_percentile = airbnb['bathrooms'].quantile(0.99)
airbnb = airbnb[airbnb['bathrooms'] <= upper_percentile]

upper_percentile = airbnb['beds'].quantile(0.99)
airbnb = airbnb[airbnb['beds'] <= upper_percentile]

upper_percentile = airbnb['bedrooms'].quantile(0.99)
airbnb = airbnb[airbnb['bedrooms'] <= upper_percentile]

upper_percentile = airbnb['number_of_reviews'].quantile(0.99)
airbnb = airbnb[airbnb['number_of_reviews'] <= upper_percentile]

columns_to_plot = ['number_of_reviews', 'bathrooms', 'bedrooms', 'beds']
plot_histograms(airbnb, columns_to_plot, figsize=(12, 8))

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(airbnb.select_dtypes(include=np.number).corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

In [ ]:
airbnb['log_number_of_reviews'] = np.log(airbnb['number_of_reviews'] + 1)
airbnb['log_accommodates'] = np.log(airbnb['accommodates'] + 1)
airbnb['squared_review_scores_rating'] = airbnb['review_scores_rating'] ** 2
columns_to_plot = ['number_of_reviews', 'log_number_of_reviews', 'accommodates', 'log_accommodates', 'review_scores_rating', 'squared_review_scores_rating']
plot_histograms(airbnb, columns_to_plot, figsize=(12, 12))

In [ ]:
plt.figure(figsize=(10, 6))
corr_matrix = airbnb[['review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'price']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap: Price and Star Ratings', fontsize=16)
plt.show()

In [ ]:
airbnb = airbnb.drop(columns = ['review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_rating'])
airbnb = airbnb.drop(columns = ['number_of_reviews', 'accommodates'])

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(airbnb.select_dtypes(include=np.number).corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

Modelling

In [ ]:
# Imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Categorical features encoding
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X_temp = airbnb.copy()

numerical_cols = X_temp.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X_temp.select_dtypes(include=['object']).columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

transformed_data = preprocessor.fit_transform(X_temp)

cat_feature_names = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_cols)
all_feature_names = numerical_cols + list(cat_feature_names)

transformed_df = pd.DataFrame(transformed_data, columns=all_feature_names)

print(f"Number of columns after encoding and scaling: {len(transformed_df.columns)}")
print(transformed_df.columns)
transformed_df

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(transformed_df.corr(), annot=False, fmt=".2f", cmap='viridis')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
# Split the data
X = transformed_df.drop(['price'], axis=1)
y = transformed_df['price']
# y = airbnb['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Select most important features
from sklearn.ensemble import RandomForestRegressor
import numpy as np

feature_selector = RandomForestRegressor(n_estimators=100, random_state=42)
feature_selector.fit(X_train, y_train)

y_pred_test = feature_selector.predict(X_test)
print("\nTest Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred_test):.2f}")

feature_importances = feature_selector.feature_importances_
important_features = np.argsort(feature_importances)[-15:]
X_train_selected = X_train.iloc[:, important_features]
X_test_selected = X_test.iloc[:, important_features]

print("Selected important features:")
print(X_train.columns[important_features])

In [ ]:
X_train_selected

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

linear_reg = LinearRegression()
linear_reg.fit(X_train_selected, y_train)

coefficients = linear_reg.coef_

coef_df = pd.DataFrame({'Feature': X_train_selected.columns, 'Coefficient': coefficients})

print("\nCoefficients for Selected Features:")
print(coef_df)

y_pred_train = linear_reg.predict(X_train_selected)
y_pred_test = linear_reg.predict(X_test_selected)

print("Training Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"R^2 Score: {r2_score(y_train, y_pred_train):.2f}")

print("\nTest Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred_test):.2f}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

random_forest = RandomForestRegressor(n_estimators=200, random_state=42)

param_grid = {
    'n_estimators': [600],
    'max_depth': [15],
    # 'min_samples_split': [5, 10],
    # 'min_samples_leaf': [4, 6, 8],
    # 'max_features': ['auto', 0.5, 0.8]
}

grid_search = GridSearchCV(estimator=random_forest,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           cv=3,
                           verbose=2,
                           n_jobs=-1)

grid_search.fit(X_train_selected, y_train)

###################################################################
cv_results = grid_search.cv_results_
best_index = grid_search.best_index_
best_train_score = -cv_results['mean_train_score'][best_index]
best_test_score = -cv_results['mean_test_score'][best_index]
best_test_std = cv_results['std_test_score'][best_index]

print("\nTraining Set Metrics (CV Results for Best Parameters):")
print(f"Mean Train MSE: {best_train_score:.4f}")
print(f"Mean CV Test MSE: {best_test_score:.4f} ± {best_test_std:.4f}")
###################################################################

best_rf = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

y_pred_train = best_rf.predict(X_train_selected)
y_pred_test = best_rf.predict(X_test_selected)

# print("Cross-Validation Metrics (Training Set):")
# print(f"Mean CV MSE: {-np.mean(cv_mse_scores):.2f} ± {np.std(cv_mse_scores):.2f}")
# print(f"Mean CV R^2: {np.mean(cv_r2_scores):.2f} ± {np.std(cv_r2_scores):.2f}")

print("\nTraining Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"R^2 Score: {r2_score(y_train, y_pred_train):.2f}")

print("\nTest Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred_test):.2f}")

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

xgb_regressor = XGBRegressor(objective='reg:squarederror', random_state=42, verbosity=1)

param_grid = {
    'n_estimators': [800],
    'max_depth': [15],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [1]
}

grid_search = GridSearchCV(estimator=xgb_regressor,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           cv=3,
                           verbose=2,
                           n_jobs=-1)

grid_search.fit(X_train_selected, y_train)

###################################################################
cv_results = grid_search.cv_results_
best_index = grid_search.best_index_
best_train_score = -cv_results['mean_train_score'][best_index]
best_test_score = -cv_results['mean_test_score'][best_index]
best_test_std = cv_results['std_test_score'][best_index]

print("\nTraining Set Metrics (CV Results for Best Parameters):")
print(f"Mean Train MSE: {best_train_score:.4f}")
print(f"Mean CV Test MSE: {best_test_score:.4f} ± {best_test_std:.4f}")
###################################################################

best_xgb = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

y_pred_train = best_xgb.predict(X_train_selected)
y_pred_test = best_xgb.predict(X_test_selected)

# scaler = preprocessor.transformers_[0][1]
# y_pred_real = scaler.inverse_transform(
#     np.concatenate([X_test_selected, y_pred_test.reshape(-1, 1)], axis=1)
# )[:, -1]
# y_real = scaler.inverse_transform(
#     np.concatenate([X_test_selected, y_test.reshape(-1, 1)], axis=1)
# )[:, -1]
# 
# # Step 5: Calculate error metrics in the original scale
# mae_real = mean_absolute_error(y_real, y_pred_real)
# mse_real = mean_squared_error(y_real, y_pred_real)
# r2_real = r2_score(y_real, y_pred_real)

# Display results
print("\nTraining Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"R^2 Score: {r2_score(y_train, y_pred_train):.2f}")

print("\nTest Set Metrics:")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_test):.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred_test):.2f}")

# print("\nTest Set Metrics scaled back:")
# print(f"Mean Squared Error (MSE): {mse_real:.2f}")
# print(f"Mean Absolute Error (MAE): {mae_real: .2f}")
# print(f"R^2 Score: {r2_real:.2f}")

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

mlp_model = MLPRegressor(
    hidden_layer_sizes=(256, 128, 64, 32),
    activation='relu',
    solver='adam',
    alpha=0.03,
    max_iter=300,
    random_state=42
)

rf_model = RandomForestRegressor(
    n_estimators=800,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=4,
    random_state=42
)

xgb_model = XGBRegressor(
    n_estimators=800,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

stacking_reg = StackingRegressor(
    estimators=[
        ('rf', rf_model),
        ('mlp', mlp_model),
        ('xgb', xgb_model)
    ],
    final_estimator=Ridge(alpha=3.0)
)

# Cross-validation
# kf = KFold(n_splits=3, shuffle=True, random_state=42)
# cv_scores = cross_val_score(stacking_model, X_train_selected, y_train, cv=kf, scoring='neg_mean_squared_error')
# 
# print(f"Cross-Validation Mean Squared Error: {-cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

stacking_reg.fit(X_train_selected, y_train)

y_pred_train = stacking_reg.predict(X_train_selected)
y_pred_test = stacking_reg.predict(X_test_selected)

print("Training Set Metrics:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}, R^2: {r2_score(y_train, y_pred_train):.2f}")

print("\nTest Set Metrics:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}, R^2: {r2_score(y_test, y_pred_test):.2f}")

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

param_grid_mlp = {
    'hidden_layer_sizes': [(256, 128, 64, 64, 32, 16), (256, 256, 256)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.01, 0.05, 0.1, 0.2]
}

grid_mlp = GridSearchCV(MLPRegressor(max_iter=4000, random_state=42), param_grid_mlp, cv=3, scoring='neg_mean_squared_error', return_train_score=True)
grid_mlp.fit(X_train_selected, y_train)

###################################################################
cv_results = grid_mlp.cv_results_
best_index = grid_mlp.best_index_
best_train_score = -cv_results['mean_train_score'][best_index]
best_test_score = -cv_results['mean_test_score'][best_index]
best_test_std = cv_results['std_test_score'][best_index]

print("\nCV Results for Best Parameters:")
print(f"Mean Train MSE: {best_train_score:.4f}")
print(f"Mean CV Test MSE: {best_test_score:.4f} ± {best_test_std:.4f}")
###################################################################
best_mlp = grid_mlp.best_estimator_

y_pred_train = best_mlp.predict(X_train_selected)
y_pred_test = best_mlp.predict(X_test_selected)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Training Set Metrics:")
print(f"MSE: {mse_train:.2f}, R^2: {r2_train:.2f}")

print("\nTest Set Metrics:")
print(f"MSE: {mse_test:.2f}, R^2: {r2_test:.2f}")

print(f"Best parameters: {grid_mlp.best_params_}")

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param_grid_svr = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],
    'epsilon': [0.001, 0.01, 0.1, 1, 5],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 1, 10],
    'degree': [2, 3, 4]
}
grid_svr = GridSearchCV(SVR(), param_grid_svr, cv=3, scoring='neg_mean_squared_error')
grid_svr.fit(X_train_selected, y_train)

best_svr = grid_svr.best_estimator_
print(f"Best Parameters: {grid_svr.best_params_}")

y_pred_train = best_svr.predict(X_train_selected)
y_pred_test = best_svr.predict(X_test_selected)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Training Set Metrics:")
print(f"MSE: {mse_train:.2f}, R^2: {r2_train:.2f}")

print("\nTest Set Metrics:")
print(f"MSE: {mse_test:.2f}, R^2: {r2_test:.2f}")

print(f"Best parameters: {grid_svr.best_params_}")

In [ ]:
best_linear_regression_rmse = 93.18
best_linear_r2 = 0.34

best_random_forest_rmse = 41.12
best_random_forest_r2 = 0.71

best_xgb_rmse = 48.32
best_xgb_r2 = 0.69

best_stacking_rmse = 29.77
best_stacking_r2 = 0.79

best_mlp_rmse = 51.37
best_mlp_r2 = 0.61

best_svr_rmse = 54.81
best_svr_r2 = 0.61

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

data = {
    'Method': ['Linear Regression', 'Random Forest', 'XGBoost', 'Stacking (XGB + RF)', 'MLP', 'SVR'],
    'RMSE': [best_linear_regression_rmse, best_random_forest_rmse, best_xgb_rmse, best_stacking_rmse, best_mlp_rmse, best_svr_rmse],
    'R2': [best_linear_r2, best_random_forest_r2, best_xgb_r2, best_stacking_r2, best_mlp_r2, best_svr_r2]
}

df = pd.DataFrame(data)

bar_width = 0.4
x = np.arange(len(df['Method']))
fig, ax1 = plt.subplots(figsize=(12, 6))

bars_rmse = ax1.bar(x - bar_width/2, df['RMSE'], width=bar_width, label='RMSE', color='blue', alpha=0.7)
ax1.set_ylabel('RMSE', fontsize=12, color='blue')
ax1.set_xlabel('Model', fontsize=12)
ax1.set_xticks(x)
ax1.set_xticklabels(df['Method'], rotation=45, ha='right')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
bars_r2 = ax2.bar(x + bar_width/2, df['R2'], width=bar_width, label='R2', color='green', alpha=0.7)
ax2.set_ylabel('R2', fontsize=12, color='green')
ax2.tick_params(axis='y', labelcolor='green')

fig.legend(loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=2, fontsize=12)
plt.title('RMSE and R2 Comparison for Different Models', fontsize=14)
plt.tight_layout()
plt.show()
